In [1]:
import os, re, datetime, bisect
from decimal import Decimal
import pandas as pd
from tqdm import tqdm
from google.colab import files

In [2]:
uploaded = files.upload()
vcd_files = [fname for fname in uploaded.keys() if fname.endswith(".vcd")]
if not vcd_files:
    raise SystemExit("No .vcd uploaded.")
vcd_path = vcd_files[0]
print("Using VCD file:", vcd_path)

Saving result.vcd to result.vcd
Using VCD file: result.vcd


In [5]:
SCOPE_FILTER = "tb.uut"   # set to None to keep all signals; set to "tb.uut" to avoid double counting with tb.*
VDD_VOLTS    = 1.0        # used by the simple power estimate
CEFF_FARADS  = 1e-15      # effective capacitance per bit (assumed)
# Power model used: P_dyn ≈ (TC_bits * Ceff * VDD^2) / duration_seconds

In [7]:
class VCDDefs:
    _factor = {"s": '1e0', "ms": '1e-3', "us": '1e-6',
               "ns": '1e-9', "ps": '1e-12', "fs": '1e-15'}

    def __init__(self, vcd_path):
        self.vcd_path = vcd_path
        self.signals = []   # list of dicts {name,id,size,type}
        self.timescale = {}

    def read_definitions(self):
        hier = []
        with open(self.vcd_path, 'r') as f:
            for line in f:
                if '$enddefinitions' in line:
                    break
                if line.startswith('$scope'):
                    hier.append(line.split()[2])
                elif line.startswith('$upscope'):
                    if hier:
                        hier.pop()
                elif line.startswith('$var'):
                    parts = line.split()
                    var_type   = parts[1]
                    size       = int(parts[2])
                    identifier = parts[3]
                    raw_name   = ''.join(parts[4:-1])   # keep [msb:lsb]
                    path = '.'.join(hier)
                    full_name = f"{path}.{raw_name}" if path else raw_name
                    self.signals.append({
                        "name": full_name,
                        "id": identifier,
                        "size": size,
                        "type": var_type
                    })
                elif line.startswith('$timescale'):
                    ts = line
                    if '$end' not in ts:
                        while True:
                            nxt = f.readline()
                            if not nxt: break
                            ts += " " + nxt.strip()
                            if '$end' in nxt: break
                    mag = Decimal(re.findall(r"\d+", ts)[0])
                    unit = re.findall(r"s|ms|us|ns|ps|fs", ts)[0]
                    self.timescale = {
                        "magnitude": mag,
                        "unit": unit,
                        "factor": Decimal(self._factor[unit]),
                        "timescale": mag * Decimal(self._factor[unit]),
                    }

v = VCDDefs(vcd_path)
v.read_definitions()
print(f"\nParsed {len(v.signals)} signals")
print("Timescale:", v.timescale or "(not found)")
print(f"Printing the signals:")
for s in v.signals[:-1]:
    print(f"  {s['name']}  (id={s['id']}, size={s['size']}, type={s['type']})")




Parsed 54 signals
Timescale: {'magnitude': Decimal('1'), 'unit': 's', 'factor': Decimal('1'), 'timescale': Decimal('1')}
Printing the signals:
  tb.clk  (id=!, size=1, type=reg)
  tb.nrst  (id=", size=1, type=reg)
  tb.state[1:0]  (id=#, size=2, type=reg)
  tb.write_enable  (id=$, size=1, type=reg)
  tb.read_enable  (id=%, size=1, type=reg)
  tb.address[11:0]  (id=&, size=12, type=reg)
  tb.write_data[31:0]  (id=', size=32, type=reg)
  tb.read_data[31:0]  (id=(, size=32, type=reg)
  tb.uut.i_clk  (id=), size=1, type=reg)
  tb.uut.i_nrst  (id=*, size=1, type=reg)
  tb.uut.i_state[1:0]  (id=+, size=2, type=reg)
  tb.uut.i_write_enable  (id=,, size=1, type=reg)
  tb.uut.i_read_enable  (id=-, size=1, type=reg)
  tb.uut.i_address[11:0]  (id=., size=12, type=reg)
  tb.uut.i_write_data[31:0]  (id=/, size=32, type=reg)
  tb.uut.o_read_data[31:0]  (id=0, size=32, type=reg)
  tb.uut.sel_d[1:0]  (id=1, size=2, type=reg)
  tb.uut.clk_enable[0:3]  (id=2, size=4, type=reg)
  tb.uut.gen_banks(0).ban

In [9]:
id_to_meta = {s["id"]: s for s in v.signals}
ids = list(id_to_meta.keys())

# Optionally scope-filter signals to avoid double counting across hierarchy
if SCOPE_FILTER:
    ids = [sid for sid in ids if id_to_meta[sid]["name"].startswith(SCOPE_FILTER)]

last_val   = {sid: None for sid in ids}
last_time  = {sid: None for sid in ids}
tc         = {sid: 0 for sid in ids}     # toggle count (scalar), or sum of bit flips (vector)
hd_sum     = {sid: 0 for sid in ids}     # total Hamming across transitions
t0         = {sid: 0 for sid in ids}     # dwell at 0 (only meaningful for 1-bit)
t1         = {sid: 0 for sid in ids}     # dwell at 1 (only meaningful for 1-bit)
tx         = {sid: 0 for sid in ids}     # dwell at X/Z or unknown
current_time = 0
begin_time = None
end_time   = 0
in_dumpvars = False

def norm_scalar(ch):
    ch = ch.strip()
    if ch in ('0','1','x','X','z','Z'):
        return ch.lower()
    return 'x'

def norm_vector(bits):
    bits = bits.strip()
    return ''.join('x' if c not in '01xzXZ' else c.lower() for c in bits)

def hamming(a, b):
    n = min(len(a), len(b))
    diff = 0
    for i in range(n):
        ca, cb = a[i], b[i]
        if ca in '01' and cb in '01' and ca != cb:
            diff += 1
    return diff

def advance_dwell(sid, new_time):
    """Accrue dwell time for the current value of sid from last_time to new_time."""
    lv = last_val[sid]
    lt = last_time[sid]
    if lt is None or lv is None:
        last_time[sid] = new_time
        return
    dt = new_time - lt
    if dt <= 0:
        return
    if id_to_meta[sid]["size"] == 1:
        if lv == '0':
            t0[sid] += dt
        elif lv == '1':
            t1[sid] += dt
        else:
            tx[sid] += dt
    else:
        if any(c not in '01' for c in lv):
            tx[sid] += dt
    last_time[sid] = new_time

with open(v.vcd_path, 'r') as f:
    for raw in f:
        line = raw.strip()
        if not line:
            continue

        if line.startswith('$'):
            if line.startswith('$dumpvars'):
                in_dumpvars = True
            elif line.startswith('$end') and in_dumpvars:
                in_dumpvars = False
            continue

        if line.startswith('#'):
            t = int(line[1:])
            if begin_time is None:
                begin_time = t
                for sid in ids:
                    last_time[sid] = begin_time
            current_time = t
            if t > end_time:
                end_time = t
            continue

        if not ids:
            continue

        c0 = line[0]
        if c0 in '01xXzZ':
            val = norm_scalar(c0)
            sid = line[1:]
            if sid not in id_to_meta or sid not in ids:
                continue
            advance_dwell(sid, current_time)
            prev = last_val[sid]
            if prev is not None:
                if prev in '01' and val in '01' and prev != val:
                    tc[sid] += 1
                    hd_sum[sid] += 1
            last_val[sid] = val
            continue

        if c0 in 'bBrR':
            try:
                payload, sid = line[1:].split()
            except ValueError:
                continue
            if sid not in id_to_meta or sid not in ids:
                continue
            bits = norm_vector(payload)
            advance_dwell(sid, current_time)
            prev = last_val[sid]
            if prev is not None:
                hd = hamming(prev, bits)
                hd_sum[sid] += hd
                tc[sid] += hd
            last_val[sid] = bits
            continue

if begin_time is None:
    begin_time = 0
for sid in ids:
    advance_dwell(sid, end_time)

# Build DataFrame
rows = []
for sid in ids:
    meta = id_to_meta[sid]
    rows.append({
        "signal": meta["name"],
        "width":  meta["size"],
        "TC":     tc[sid],
        "HD_sum": hd_sum[sid],
        "T0":     t0[sid],
        "T1":     t1[sid],
        "TX":     tx[sid],
        "begin_time": begin_time,
        "end_time":   end_time,
        "timescale_unit": v.timescale.get("unit","unknown"),
    })
df_vcd = pd.DataFrame(rows)
csv_name = "switching_activity_vcd.csv"
df_vcd.to_csv(csv_name, index=False)
print(f"\nSaved VCD activity CSV: {csv_name}")
display(df_vcd.head())



Saved VCD activity CSV: switching_activity_vcd.csv


,signal,width,TC,HD_sum,T0,T1,TX,begin_time,end_time,timescale_unit
0,tb.uut.i_clk,1,200000,200000,500000000000,500000000000,0,0,1000000000000,s
1,tb.uut.i_nrst,1,1,1,20000000,999980000000,0,0,1000000000000,s
2,tb.uut.i_state[1:0],2,10,10,0,0,0,0,1000000000000,s
3,tb.uut.i_write_enable,1,8,8,999780000000,220000000,0,0,1000000000000,s
4,tb.uut.i_read_enable,1,4,4,999780000000,220000000,0,0,1000000000000,s


In [10]:
def build_hierarchy(names):
    """Build a nested dict for SAIF (INSTANCE tree) from hierarchical net names."""
    root = {"children": {}, "nets": set()}
    for full in names:
        parts = full.split('.')
        node = root
        for p in parts[:-1]:
            node = node["children"].setdefault(p, {"children": {}, "nets": set()})
        node["nets"].add(parts[-1])
    return root

def write_saif(saif_path, design_top, activity, duration_ticks, timescale):
    """
    SAIF v2.0 writer:
      (TIMESCALE <magnitude> <unit>)
      (DURATION <ticks>)
      per-net: (NET <name> (T0 ..) (T1 ..) (TX ..) (TC ..) (IG 0))
    """
    mag = int(timescale.get("magnitude", Decimal("1")))
    unit = timescale.get("unit", "ns")
    now  = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    hier = build_hierarchy(activity.keys())

    def emit_instance(fp, inst_name, node, indent):
        ind = ' ' * indent
        fp.write(f"{ind}(INSTANCE {inst_name}\n")

        # emit nets directly under this instance
        for net in sorted(node["nets"]):
            # Find best matching activity key (full hierarchical name)
            # Prefer exact "inst_name.net", otherwise find any that endswith(".net")
            key_candidates = [k for k in activity.keys()
                              if k == f"{inst_name}.{net}" or k.endswith(f".{inst_name}.{net}") or k.endswith(f".{net}")]
            key = None
            if f"{inst_name}.{net}" in activity:
                key = f"{inst_name}.{net}"
            elif key_candidates:
                key = key_candidates[-1]
            vals = activity.get(key, {"T0":0,"T1":0,"TX":0,"TC":0})
            fp.write(f"{ind}  (NET {net} (T0 {vals['T0']}) (T1 {vals['T1']}) (TX {vals['TX']}) (TC {vals['TC']}) (IG 0))\n")

        # recurse for children
        for child_name in sorted(node["children"].keys()):
            emit_instance(fp, child_name, node["children"][child_name], indent+2)

        fp.write(f"{ind})\n")

    with open(saif_path, "w") as fp:
        fp.write("(SAIFILE\n")
        fp.write('  (SAIFVERSION "2.0")\n')
        fp.write("  (DIRECTION FROM_TOP_TO_BOTTOM)\n")
        fp.write(f'  (DESIGN "{design_top}")\n')
        fp.write(f'  (DATE "{now}")\n')
        fp.write('  (VENDOR "OpenAI")\n')
        fp.write('  (PROGRAM_NAME "vcd2saif_py")\n')
        fp.write('  (VERSION "0.1")\n')
        fp.write(f"  (TIMESCALE {mag} {unit})\n")
        fp.write(f"  (DURATION {duration_ticks})\n")
        fp.write("  ")
        emit_instance(fp, design_top, hier, indent=2)
        fp.write(")\n")


# Prepare activity dict (using the *filtered* df_vcd)
activity = {}
for _, r in df_vcd.iterrows():
    activity[r["signal"]] = {"T0": int(r["T0"]), "T1": int(r["T1"]), "TX": int(r["TX"]), "TC": int(r["TC"])}

# Choose design top as first token
if len(df_vcd) and '.' in df_vcd.iloc[0]["signal"]:
    design_top = df_vcd.iloc[0]["signal"].split('.')[0]
else:
    design_top = "TOP"

duration_ticks = max(0, int((end_time or 0) - (begin_time or 0)))
saif_name = os.path.splitext(os.path.basename(vcd_path))[0] + ".saif"
write_saif(saif_name, design_top, activity, duration_ticks, v.timescale)
print(f"\nSAIF written: {saif_name}  (TIMESCALE {v.timescale.get('magnitude','1')} {v.timescale.get('unit','ns')}, DURATION {duration_ticks})")
files.download(saif_name)


SAIF written: result.saif  (TIMESCALE 1 s, DURATION 1000000000000)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
def parse_saif(path):
    """
    Very simple SAIF reader:
    returns list of rows: {"fullpath": "...", "net": "...", "T0":.., "T1":.., "TX":.., "TC":..}
    """
    rows = []
    stack = []   # instance name stack
    with open(path, "r") as fp:
        for raw in fp:
            line = raw.strip()
            if not line: continue
            if line.startswith("(INSTANCE "):
                name = line.split()[1]
                stack.append(name)
            elif line == ")":
                if stack: stack.pop()
            elif line.startswith("(NET "):
                # (NET netname (T0 X) (T1 Y) (TX Z) (TC W) (IG 0))
                m = re.findall(r"\(NET\s+([^\s]+)\s+\(T0\s+(\d+)\)\s+\(T1\s+(\d+)\)\s+\(TX\s+(\d+)\)\s+\(TC\s+(\d+)\)", line)
                if m:
                    net, T0s, T1s, TXs, TCs = m[0]
                    full = ".".join(stack+[net]) if stack else net
                    rows.append({
                        "signal": full,
                        "T0": int(T0s), "T1": int(T1s), "TX": int(TXs), "TC": int(TCs)
                    })
    return pd.DataFrame(rows)

df_saif = parse_saif(saif_name)
print("\nParsed SAIF back to DataFrame (first 12 rows):")
display(df_saif.head())


Parsed SAIF back to DataFrame (first 12 rows):


,signal,T0,T1,TX,TC
0,tb.tb.uut.clk_enable[0:3],0,0,5000000,26
1,tb.tb.uut.i_address[11:0],0,0,0,54
2,tb.tb.uut.i_clk,500000000000,500000000000,0,200000
3,tb.tb.uut.i_nrst,20000000,999980000000,0,1
4,tb.tb.uut.i_read_enable,999945000000,50000000,0,4


In [12]:
# Inner join on signal names
cmp = pd.merge(df_vcd[["signal","TC","T0","T1","TX"]],
               df_saif[["signal","TC","T0","T1","TX"]],
               on="signal", suffixes=("_vcd","_saif"))

cmp["dTC"] = cmp["TC_vcd"] - cmp["TC_saif"]
cmp["dT0"] = cmp["T0_vcd"] - cmp["T0_saif"]
cmp["dT1"] = cmp["T1_vcd"] - cmp["T1_saif"]
cmp["dTX"] = cmp["TX_vcd"] - cmp["TX_saif"]

print("\nVCD vs SAIF compare (first 20 rows):")
display(cmp.head(20))

mismatches = cmp[(cmp["dTC"]!=0) | (cmp["dT0"]!=0) | (cmp["dT1"]!=0) | (cmp["dTX"]!=0)]
print(f"Total mismatched nets: {len(mismatches)} (should be 0 if writer/reader round-trip is consistent)")
display(mismatches.head())


VCD vs SAIF compare (first 20 rows):


,signal,TC_vcd,T0_vcd,T1_vcd,TX_vcd,TC_saif,T0_saif,T1_saif,TX_saif,dTC,dT0,dT1,dTX


Total mismatched nets: 0 (should be 0 if writer/reader round-trip is consistent)


,signal,TC_vcd,T0_vcd,T1_vcd,TX_vcd,TC_saif,T0_saif,T1_saif,TX_saif,dTC,dT0,dT1,dTX


In [13]:
duration_seconds = float(duration_ticks) * float(v.timescale.get("timescale", Decimal("1e-9")))
if duration_seconds <= 0:
    duration_seconds = 1.0  # fallback, avoids divide-by-zero

df_power = df_vcd.copy()
# If width>1 we already counted TC as sum of bit-toggles; assume Ceff per bit is constant
df_power["Energy_J"] = df_power["TC"] * CEFF_FARADS * (VDD_VOLTS**2)
df_power["Power_W"]  = df_power["Energy_J"] / duration_seconds

print("\nEstimated dynamic power per net (top 20 by power):")
display(df_power.sort_values("Power_W", ascending=False).head(20))

# Save results too
df_vcd.to_csv("switching_activity_vcd.csv", index=False)
df_saif.to_csv("switching_activity_saif.csv", index=False)
df_power.to_csv("power_estimate_vcd.csv", index=False)
print("\nSaved: switching_activity_vcd.csv, switching_activity_saif.csv, power_estimate_vcd.csv")


Estimated dynamic power per net (top 20 by power):


,signal,width,TC,HD_sum,T0,T1,TX,begin_time,end_time,timescale_unit,Energy_J,Power_W
0,tb.uut.i_clk,1,200000,200000,500000000000,500000000000,0,0,1000000000000,s,2.000000e-10,2.000000e-22
10,tb.uut.gen_banks(0).bank.i_clk,1,200000,200000,500000000000,500000000000,0,0,1000000000000,s,2.000000e-10,2.000000e-22
28,tb.uut.gen_banks(2).bank.i_clk,1,200000,200000,500000000000,500000000000,0,0,1000000000000,s,2.000000e-10,2.000000e-22
37,tb.uut.gen_banks(3).bank.i_clk,1,200000,200000,500000000000,500000000000,0,0,1000000000000,s,2.000000e-10,2.000000e-22
19,tb.uut.gen_banks(1).bank.i_clk,1,200000,200000,500000000000,500000000000,0,0,1000000000000,s,2.000000e-10,2.000000e-22
6,tb.uut.i_write_data[31:0],32,393,393,0,0,0,0,1000000000000,s,3.930000e-13,3.930000e-25
26,tb.uut.gen_banks(1).bank.i_write_data[31:0],32,393,393,0,0,0,0,1000000000000,s,3.930000e-13,3.930000e-25
17,tb.uut.gen_banks(0).bank.i_write_data[31:0],32,393,393,0,0,0,0,1000000000000,s,3.930000e-13,3.930000e-25
44,tb.uut.gen_banks(3).bank.i_write_data[31:0],32,393,393,0,0,0,0,1000000000000,s,3.930000e-13,3.930000e-25
35,tb.uut.gen_banks(2).bank.i_write_data[31:0],32,393,393,0,0,0,0,1000000000000,s,3.930000e-13,3.930000e-25



Saved: switching_activity_vcd.csv, switching_activity_saif.csv, power_estimate_vcd.csv
